In [18]:
from keras.models import Sequential , Model
from keras.layers import Dense , Activation
from keras.layers import Dropout , GlobalAveragePooling2D
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD , RMSprop , Adadelta , Adam
from keras.layers import Conv2D , BatchNormalization
from keras.layers import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt

In [19]:
base_model=InceptionV3(weights='imagenet',include_top=False,input_shape=(3, 150, 150)) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(2,activation='softmax')(x) #final layer with softmax activation

In [20]:
model = Model(inputs=base_model.input, outputs=preds)

In [21]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 3, 150, 150)  0                                            
__________________________________________________________________________________________________
conv2d_189 (Conv2D)             (None, 32, 74, 74)   864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_189 (BatchN (None, 32, 74, 74)   96          conv2d_189[0][0]                 
__________________________________________________________________________________________________
activation_189 (Activation)     (None, 32, 74, 74)   0           batch_normalization_189[0][0]    
__________________________________________________________________________________________________
conv2d_190

In [24]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('/home/abu/Desktop/resp/b_cancer_data2/train', # this is where you specify the path to the main data folder
                                                 target_size=(150,150),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

test_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

test_generator=test_datagen.flow_from_directory('/home/abu/Desktop/resp/b_cancer_data2/test', # this is where you specify the path to the main data folder
                                                 target_size=(150,150),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 6345 images belonging to 2 classes.
Found 1564 images belonging to 2 classes.


In [25]:
step_size_train=train_generator.n//train_generator.batch_size
step_size_test=test_generator.n//test_generator.batch_size


In [26]:
history=model.fit_generator(generator=train_generator,steps_per_epoch=step_size_train,
                            validation_data=test_generator,validation_steps=step_size_test,epochs=3)

Epoch 1/3
  2/198 [..............................] - ETA: 1:23:46 - loss: 3.0777 - acc: 0.4062

KeyboardInterrupt: 

In [ ]:
from keras.models import load_model
# Creates a HDF5 file 'my_model.h5'
model.save('transfer_cancer.h5')

In [ ]:
#val accuracy
score  = model.evaluate_generator(test_generator, steps=len(test_generator), verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])